In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\diamond_price_predication_2\\research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\diamond_price_predication_2'

In [5]:
import pandas as pd
data=pd.read_csv("artifacts\data_ingestion\diamond_data.csv")

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


In [7]:
data.dtypes

id           int64
carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
x          float64
y          float64
z          float64
price        int64
dtype: object

In [8]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataValidationConfig: 
    root_dir: Path
    unzip_dir: Path
    status_file: Path
    all_schema: dict

In [14]:
from diamond_price_predication.constant import *
from diamond_price_predication.utils.common import read_yaml, create_dicectories
from diamond_price_predication import logger

In [32]:
class ConfigurationManger:
    def __init__(self, 
                 config_file_path= CONFIG_FILE_PATH, 
                 params_file_path= PARAMS_FILE_PATH, 
                 schema_file_path= SCHEMA_FILE_PATH
                 ):
        
        self.config=read_yaml(config_file_path)
        self. params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        
        create_dicectories([self.config.artifacts_root])
        
    def get_validation_config(self)-> DataValidationConfig: 
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        
        create_dicectories([config.root_dir])
        
        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            unzip_dir=config.unzip_data_dir, 
            status_file=config.STATUS_FILE,
            all_schema=schema
        )
        return data_validation_config

In [35]:
class DataValidation: 
    def __init__(self, config: DataValidationConfig): 
        self.config=config
        
        
        
    def validate_all_columns(self)->bool: 
        try: 
            validation_status: None
            data=pd.read_csv(self.config.unzip_dir)
            all_cols=list(data.columns)
            logger.info(f" Columns: {all_cols}")
            
            all_schema=self.config.all_schema.keys()
            
            logger.info(f"All schema {all_schema}")
            
            for col in all_cols: 
                if col not in all_schema: 
                    
                    
                    validation_status=False
                    
                    with open(self.config.status_file, 'w') as f: 
                        f.write(f"Validation status: {validation_status}")
                    
            else: 
                validation_status=True
                with open(self.config.status_file,'w') as f: 
                    f.write(f" Validation status: {validation_status}")
                    
            return validation_status
            
            
        
        except Exception as e: 
            raise e

In [36]:
try: 
    obj=ConfigurationManger()
    validationConfig=obj.get_validation_config()
    datavalidation=DataValidation(validationConfig)
    datavalidation.validate_all_columns()
except Exception as e: 
    raise e

[2023-12-11 17:07:02,679 : INFO: common : Yaml file read config\config.yaml]
[2023-12-11 17:07:02,684 : INFO: common : Yaml file read params.yaml]
[2023-12-11 17:07:02,694 : INFO: common : Yaml file read schema.yaml]
[2023-12-11 17:07:02,698 : INFO: common : Directories file is creted artifacts]
[2023-12-11 17:07:02,701 : INFO: common : Directories file is creted artifacts/data_validation]
[2023-12-11 17:07:03,410 : INFO: 214939789 :  Columns: ['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price']]
[2023-12-11 17:07:03,413 : INFO: 214939789 : All schema dict_keys(['id', 'carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price'])]
